### Backtest Framework
This notebook is the most important foundation work designed specifically for cbonvertible bond strategy research.

It provides high-level API to interact with your Strategy class. 

**Basic Introduction**:
+ Set backtesting parameters in python file `backtest_config` 
+ Trading at both open and close time are allowed
+ Two main functions provided for Strategy class: `order` and `transfer`
+ Imported and used mainly in `Strategy.ipynb`

**Backtest Steps**:
1. Initialize backtest system with config
2. Call backtest pipeline with Srategy class (defined in `Strategy.ipynb`)
3. Backtest analysis

**Sample code for usage**:
```python
# import packages and Backtest.ipynb
import import_ipynb
import backtest_config
from Backtest import CbondBacktest

# parent Strategy class 
class Strategy:
    def __init__(self, CBT):
        self.CBT = CBT
    # =============== Interaction with Backtest System =================
    def sendOrder(self, date, time='close'):
        pass
    
    def analysis(self):
        pass

# 4 lines code to backtest your strategy
CBT = CbondBacktest(backtest_config)   # initialize Convertible Bond Backtest system
strategy = Strategy(CBT)               # initialize strategy class 
CBT.pipeline(strategy)                 # strategy backtesting
CBT.analysis()                         # strategy performance analysis
```

In [1]:
import os
import copy
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt 

plt.style.use('seaborn')
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [1]:
class CbondBacktest:
    
    def __init__(self, config):
        # config initializing and data loading
        self.data_path = config.data_path   
        self.start_date = config.start_date
        self.end_date = config.end_date  
        self.times = config.times
        self.dataLoader() 
        
        self.trade_dates = [date for date in self.cbond_data_raw['date'].unique().tolist()
                            if date >= self.start_date and date <= self.end_date]
        
        # basic info
        self.cbond_tickers = config.cbond_tickers
        self.cbond_stock_map = dict(self.cbond_info[['bond_ticker', 'stock_ticker']].values)
        self.cbond_lastday = dict(self.cbond_data_raw.loc[self.cbond_data_raw['is_last']==True, 
                                                   ['bond_ticker', 'date']].values)  # valid last trading day
        self.longshort = config.longshort
        self.min_share = {
            'bond': config.cbond_min_share,
            'stock': config.stock_min_share 
        }
        self.valid_both = config.valid_both
        self.fee_rate = config.fee_rate
        
        # portfolio settings
        self.cash = config.capital                 # available cash (for long position)
        self.cap = config.capital                  # fixed initial capital
        
        # backtest info record 
        self.portfolio = {     
            'bond':{}, 
            'stock':{}
        }                                          # updated portfolio
        self.portfolio_record = {}                 # historical portfolio record, date(str): portfolio(dict)
        self.order_record = {}                     # historical excuted order record 
        self.order_received = {}                   # historical received order record
        self.transfer_record = {}                  # historical convert order record
        
        self.pnl = {}                              # historical pnl of portfolio
        self.fee = {}                              # historical transaction fee
        self.capital = {}                          # historical daily capital (cash + asset)
        
        # performance analysis param
        self.annual_date = config.annual_date      # trading day number in one year
        self.riskfree_rate = config.riskfree_rate  # annually risk free rate
        
    def dataLoader(self):
        """
        Load data 
        """
        self.cbond_info = pd.read_csv(os.path.join(self.data_path, 'cbond_info.csv'), index_col=False)
        self.cbond_data_raw = pd.read_csv(os.path.join(self.data_path, 'cbond_price.csv'), index_col=False)
        self.stock_data_raw = pickle.load(open(os.path.join(self.data_path, 'stock_data_raw.pkl'), 'rb'))
        self.cbond_data = pickle.load(open(os.path.join(self.data_path, 'cbond_data.pkl'), 'rb'))
        self.stock_data = pickle.load(open(os.path.join(self.data_path, 'stock_data.pkl'), 'rb'))
        
        self.cbond_info['start_date'] = self.cbond_info['start_date'].astype('str').map(lambda x: x.replace('-', ''))
        self.cbond_info['exit_date'] = self.cbond_info['exit_date'].astype('str').map(lambda x: x.replace('-', ''))
        self.cbond_info['conv_date'] = self.cbond_info['conv_date'].astype('str').map(lambda x: x.replace('-', ''))
        self.cbond_data_raw['date'] = self.cbond_data_raw['date'].astype('str').map(lambda x: x.replace('-', ''))

        print('== backtest init ==')
            
    
    # ===================================== Tools Part =======================================
    def getPrevDate(self, date, win=1):
        """
        get previous date
        """
        idx = self.trade_dates.index(date)
        
        if idx - win >= 0:
            return self.trade_dates[idx - win]
        
        else:
            return None
        
    
    def getNextDate(self, date, win=1):
        """
        get next date
        """
        idx = self.trade_dates.index(date)
        
        if idx + win < len(self.trade_dates):
            return self.trade_dates[idx + win]
        
        else:
            return None
        
        
    def isLastDate(self, date, bond_ticker):
        """
        check whether the date is the last trading day
        """
        return (self.cbond_lastday[bond_ticker] == date)
    
    
    def isTradeDate(self, date, ticker, asset_type):
        """
        whether the ticker is in dataset
        """
        if asset_type == 'stock':
            return ticker in self.stock_data[date].keys()
        
        else:
            return ticker in self.cbond_data[date].keys()
        
    
    def isLimit(self, date, ticker, amount, asset_type, time):
        """
        whether up/down limit at given time
        """
        if asset_type == 'stock':
            if np.sign(amount) == 1:
                direction = 'buy'
            else:
                direction = 'sell'
            is_limit = self.stock_data[date][ticker]['limit_{}_{}'.format(
                direction, time)] or self.stock_data[date][ticker]['is_unusual']
            return is_limit
        else:
            return False
    
    
    def isValidTicker(self, date, ticker, asset_type, time):
        """
        check whether the ticker is available in the market
        """
        available_bond_tickers = list(self.cbond_data[date].keys())
        available_stock_tickers = list(self.stock_data[date].keys())
        
        if asset_type == 'bond':
            stock_ticker = self.getCbondStock(ticker)
            bond_ticker = ticker
        else:
            stock_ticker = ticker
            bond_ticker = self.getStockCbond(ticker, date)
        
        # bond valid
        if bond_ticker not in available_bond_tickers:
            bond_valid = False
        else:
            cond_1 = not self.isLastDate(date, bond_ticker)
            cond_2 = ticker in self.portfolio[asset_type].keys()
            bond_valid = cond_1 | cond_2
        
        # stock valid
        if stock_ticker not in available_stock_tickers:
            stock_valid = False
        else:
            stock_valid = True
        
        if self.valid_both:
            return bond_valid & stock_valid
        else:
            if asset_type == 'stock':
                return stock_valid
            else:
                return bond_valid
    
    
    def isLongShort(self, date, ticker, amount, asset_type):
        """
        check whether valid or not to buy/sell
        """
        if int(np.sign(amount)) in self.longshort[asset_type]:
            return True
        
        elif self.isHolding(ticker, asset_type):  # 减仓/清仓时没有限制
            if np.abs(self.portfolio[asset_type][ticker]['amount']) >= np.abs(amount):
                return True
            else:
                return False
        
        else:
            return False
        
        
    def isHolding(self, ticker, asset_type):
        """
        check ticker in portfolio or not
        """
        return ticker in list(self.portfolio[asset_type].keys())
    
    
    def isOppDirection(self, ticker, amount, asset_type):
        """
        whether the order is opposite direction to current portfolio holding
        """
        if self.isHolding(ticker, asset_type):
            holding_dir = np.sign(self.portfolio[asset_type][ticker]['amount'])
            order_dir = np.sign(amount)
            if holding_dir != order_dir:
                return True
            else:
                return False
        else:
            return False
        
    
    def getValidTickers(self, date, tickers, asset_type, time):
        """
        return valid tickers available in the martket
        """
        valid_tickers = [tkr for tkr in tickers if self.isValidTicker(date, tkr, asset_type, time)]

        return valid_tickers
        
        
    def getPrice(self, date, ticker, asset_type, time):
        """
        return ticker price
        """
        if asset_type == 'bond':
            price = self.cbond_data[date][ticker]['{}'.format(time)] 
            
        else:
            price = self.stock_data[date][ticker]['adj_{}'.format(time)] 
            
        return price
    
    
    def getValue(self, date, ticker, asset_type, time):
        """
        return ticker value
        """
        if asset_type == 'bond':
            value = self.cbond_data[date][ticker]['amount'] / 1e5
            
        else:
            value = self.stock_data[date][ticker]['value'] / 1e8
            
        return value
    
    
    def getCbondStock(self, bond_ticker):
        """
        return the stock ticker of given bond ticker
        """
        return self.cbond_stock_map[bond_ticker]
    
    
    def getStockCbond(self, stock_ticker, date):
        """
        return bond ticker of the given stock ticker
        """
        dat = self.cbond_info.loc[(self.cbond_info['stock_ticker']==stock_ticker) & 
                                  (self.cbond_info['exit_date'] >= date), 'bond_ticker']
        if len(dat) == 0:
            print('No bond found {} {}'.format(date, stock_ticker))
            return None
        bond_ticker = dat.values[0]
        return bond_ticker
    
    
    def getTransferAmount(self, date, bond_ticker, amount, time):
        """
        return transferred stock amount given bond amount
        """
        stock_ticker = self.getCbondStock(bond_ticker)
        bond_price = self.getPrice(date, bond_ticker, asset_type='bond', time=time)
        stock_price = self.getPrice(date, stock_ticker, asset_type='stock', time=time)

        conv_share = self.cbond_data[date][bond_ticker]['adj_conv_share']
        stock_amount = int(amount * conv_share // self.min_share['stock'] * self.min_share['stock']) 
        return stock_amount
    
    
    def getPortfolioTypeCount(self, asset_type):
        """
        return certain asset type portfolio count
        """
        count = len(self.portfolio[asset_type])
        return count
    
    
    def getPortfolioCount(self):
        """
        return stock and bond portfolio count
        """
        return self.getPortfolioTypeCount('bond') + self.getPortfolioTypeCount('stock')
    
    
    def getPortfolioTypeValue(self, asset_type):
        """
        return certain asset type portfolio value
        """
        if self.portfolio[asset_type]: 
            value = np.sum([self.portfolio[asset_type][tkr]['value'] for tkr in self.portfolio[asset_type].keys()])
        else:
            value = 0
        
        return value
    
    
    def getPortfolioValue(self):
        """
        return stock and bond portfolio value
        """
        return self.getPortfolioTypeValue('bond') + self.getPortfolioTypeValue('stock')
    
    
    def getTotalValue(self):
        """
        return total value 
        """
        return self.getPortfolioValue() + self.cash
    
    
    def getTotalCost(self):
        """
        return portfolio total cost 
        """
        cost = 0
        for asset_type in ['stock', 'bond']:
            if self.portfolio[asset_type]: 
                costs = [self.portfolio[asset_type][tkr]['cost'] for tkr in self.portfolio[asset_type].keys()]
                cost += np.sum(costs)
            else:
                cost += 0
        
        return cost
    
    
    def getPortfolioLongCost(self):
        """
        return portfolio long position cost
        """
        long_cost = 0
        for asset_type in ['stock', 'bond']:
            if self.portfolio[asset_type]: 
                costs = [self.portfolio[asset_type][tkr]['cost'] for tkr in self.portfolio[asset_type].keys()]
                long_cost += np.sum([cost for cost in costs if cost > 0])
            else:
                long_cost += 0
        
        return long_cost
    
    
    def getTotalPNL(self, date, time):
        """
        return portfolio total holding pnl
        """
        pnl = 0
        for asset_type in ['bond', 'stock']:
            for ticker in list(self.portfolio_record[date][time][asset_type].keys()):   
                pnl += self.portfolio_record[date][time][asset_type][ticker]['pnl']
        return pnl
    
    
    def getBookPNL(self, date, time):
        """
        get last period book pnl(prev close to open, or open to close)
        """
        if time == 'close':
            return self.getTotalPNL(date, 'close') - self.getTotalPNL(date, 'open')
        
        else:
            prev_date = self.getPrevDate(date)
            
            if prev_date is not None:
                return self.getTotalPNL(date, 'open') - self.getTotalPNL(prev_date, 'close')
            else:
                return self.getTotalPNL(date, 'open')
    
    
    def getIndexRet(self, date, time):
        """
        get bond index(top1500) return
        """
        if time == 'open':
            return self.cbond_data_raw.loc[(self.cbond_data_raw['date']==date) & 
                                            (self.cbond_data_raw['top1500']==True), 'ovnt_ret'].mean()
        else:
            return self.cbond_data_raw.loc[(self.cbond_data_raw['date']==date) & 
                                           (self.cbond_data_raw['top1500']==True), 'intra_ret'].mean()
        
        
    # ===================================== Order Part =======================================
    def order(self, date, ticker, amount, asset_type, time):
        """
        basic order function:
        called in Strategy class
        """
        # record the received order infomation
        self.order_received[date].append({'date': date, 'ticker':ticker, 
                                          'amount':amount, 'asset_type':asset_type, 'time':time})
        
        # min order limit
        assert(amount % self.min_share[asset_type] == 0), (date, ticker, amount, time)
        
        # order validation check
        cond_1 = (amount == 0)
        cond_2 = not self.isValidTicker(date, ticker, asset_type, time)
        cond_3 = not self.isLongShort(date, ticker, amount, asset_type)
        if cond_1 or cond_2 or cond_3:
            return None
        
        # up or down limit 
        if self.isLimit(date, ticker, amount, asset_type, time):
            return None
        
        # order calculation
        price = self.getPrice(date, ticker, asset_type, time)
        cost = amount * price
        
        # check cash enough or not
        if asset_type=='bond' and cost > (self.cap - self.getPortfolioLongCost()):
            return None

        fee = np.abs(amount) * price * self.fee_rate[asset_type]
        
        # lending fee
        if asset_type == 'stock' and amount < 0:
            if ticker not in self.portfolio['stock'].keys():   # new position
                lending_amount = amount                        # lending amount
                selling_amount = 0                             # selling amount of long position
            else:
                if self.portfolio['stock'][ticker]['amount'] < 0: # short position in portfoliio
                    lending_amount = amount
                    selling_amount = 0
                else:                                             # long position in portfolio
                    lending_amount = min(self.portfolio['stock'][ticker]['amount'] + amount, 0)
                    selling_amount = amount - lending_amount
            
            fee += np.abs(lending_amount) * price * self.fee_rate['lending'] / self.annual_date #　annual lending cost
            fee += np.abs(selling_amount) * price * self.fee_rate['selling']      # selling cost of stock long position (tax)
            
        # cost & pnl updating 
        if self.isOppDirection(ticker, amount, asset_type):  # cut or clear position　
            init_cost = self.portfolio[asset_type][ticker]['cost'] * amount / self.portfolio[
                asset_type][ticker]['amount']
            trade_pnl = self.portfolio[asset_type][ticker]['pnl'] * np.abs(amount / self.portfolio[
                asset_type][ticker]['amount'])         
            label = 'reduce'                                 # order label
            
            self.cash -= init_cost                           # calculate pnl and initiate the cash 
            self.pnl[date][time]['trade'] += trade_pnl       # record trading pnl 
            
        else:                                                # add or open position
            init_cost = cost
            trade_pnl = 0
            self.cash -= cost
            label = 'add'
            
        self.fee[date][time] += fee
        
        # portfolio updating
        if ticker in self.portfolio[asset_type].keys():
            self.portfolio[asset_type][ticker]['amount'] += amount
            self.portfolio[asset_type][ticker]['cost'] += init_cost
            self.portfolio[asset_type][ticker]['value'] += cost
            self.portfolio[asset_type][ticker]['pnl'] = self.portfolio[
                asset_type][ticker]['value'] - self.portfolio[asset_type][ticker]['cost']
                        
            if self.portfolio[asset_type][ticker]['amount'] == 0:
                del self.portfolio[asset_type][ticker]       # amount comes to 0, delete the ticker in portfolio
                label = 'clean'
            
        else:                                                # open position, default portfolio information
            self.portfolio[asset_type][ticker] = {'amount': amount, 'cost': cost, 
                                                  'value':cost, 'pnl':0, 'time': time, 'count':0}
            label = 'build'                
        
        # record the order infomation
        order_info = {'date': date,'ticker': ticker,'asset_type':asset_type ,'dir':np.sign(amount), 
                      'amount': np.abs(amount),'price': price, 'cost': np.abs(cost), 
                      'trade_pnl':trade_pnl, 'time':time, 'label':label}
        
        self.order_record[date].append(order_info)

    
    def clearTicker(self, date, ticker, asset_type, time):
        """
        order function:
        clear all the position of given ticker in the portfolio
        """
        if ticker in self.portfolio[asset_type].keys():
            amount = self.portfolio[asset_type][ticker]['amount']
            self.order(date, ticker, -1 * amount, asset_type, time)
            
        
    def clearHolding(self, date, time):
        """
        order function:
        clear all of the holdings in the portfolio
        """
        for asset_type in self.portfolio:
            for ticker in list(self.portfolio[asset_type].keys()):
                self.clearTicker(date, ticker, asset_type, time)
    
    
    def transfer(self, date, bond_ticker, amount, time):
        """
        transfer function, similar to order function
        transfer bond holdings to stock
        Note: amount should be a positive number(reduce bond, add stock)
        """
        assert bond_ticker in self.portfolio['bond'].keys()
        
        if not self.isValidTicker(date, bond_ticker, 'bond', time):
            return None
        
        if amount > self.portfolio['bond'][bond_ticker]['amount']: # not enough bond to convert
            return None
        
        if self.getTransferAmount(date, bond_ticker, amount, time) == 0:  # not research minimum conversion amount
            return None
        
        #bond calculation
        init_cost = self.portfolio['bond'][bond_ticker]['cost'] * amount / self.portfolio[
            'bond'][bond_ticker]['amount']
        value = amount * self.getPrice(date, bond_ticker, 'bond', time)
              
        # portfolio update
        self.portfolio['bond'][bond_ticker]['amount'] -= amount
        self.portfolio['bond'][bond_ticker]['cost'] -= init_cost
        self.portfolio['bond'][bond_ticker]['value'] -= value
        self.portfolio['bond'][bond_ticker]['pnl'] = self.portfolio[
                'bond'][bond_ticker]['value'] - self.portfolio['bond'][bond_ticker]['cost']
        
        if self.portfolio['bond'][bond_ticker]['amount'] == 0:
            del self.portfolio['bond'][bond_ticker]
        
        # stock calculation
        stock_ticker = self.getCbondStock(bond_ticker)
        trans_amount = np.abs(self.getTransferAmount(date, bond_ticker, amount, time))
        trans_price = self.getPrice(date, stock_ticker, 'stock', time)
        trans_cost = init_cost   
        trans_value = trans_amount * trans_price
        trans_pnl = trans_value - trans_cost                    
        
        # pnl update
        if self.isOppDirection(stock_ticker, trans_amount, 'stock'):  # reduce or clear the position
            net_cost = self.portfolio['stock'][stock_ticker]['cost'] * np.abs(
                trans_amount / self.portfolio['stock'][stock_ticker]['amount']) + trans_cost
            transfer_pnl = self.portfolio['stock'][stock_ticker]['pnl'] * np.abs(
                trans_amount / self.portfolio['stock'][stock_ticker]['amount']) + trans_pnl
            
            self.cash += net_cost
            self.pnl[date][time]['transfer'] += transfer_pnl
        
        else:
            transfer_pnl = 0
            
        self.transfer_record[date].append({'date':date, 'bond_ticker': bond_ticker, 'stock_ticker':stock_ticker,
                                           'bond_amount':amount, 'stock_amount':trans_amount,'transfer_pnl': transfer_pnl}) 
            
        # portfolio update
        if stock_ticker in self.portfolio['stock'].keys():
            prev_amount = self.portfolio['stock'][stock_ticker]['amount']
            curr_amount = prev_amount + trans_amount
            self.portfolio['stock'][stock_ticker]['amount'] = curr_amount
            self.portfolio['stock'][stock_ticker]['cost'] = self.portfolio['stock'][stock_ticker]['cost'] * np.abs(curr_amount / prev_amount) 
            self.portfolio['stock'][stock_ticker]['value'] = self.portfolio['stock'][stock_ticker]['value'] * np.abs(curr_amount / prev_amount) 
            self.portfolio['stock'][stock_ticker]['pnl'] = self.portfolio[
                'stock'][stock_ticker]['value'] - self.portfolio['stock'][stock_ticker]['cost']
            
            if self.portfolio['stock'][stock_ticker]['amount'] == 0:
                del self.portfolio['stock'][stock_ticker]
        
        else:
            self.portfolio['stock'][stock_ticker] = {'amount': trans_amount, 'cost':trans_cost,
                                                     'value': trans_value, 'pnl':trans_pnl, 'time':time, 'count':0}
            
            
    # ===================================== Pipeline Part =======================================
    def pipeline(self, strategy):
        """
        Pipeline for backtesting
        strategy: Strategy<class> defined to interact with BT system. Must have sendOrder(date, time) function
        basic pipeline: update daily portfolio status & pnl -> call Strategy class -> caculate pnl
        """
        for date in tqdm(self.trade_dates):
            # initialize different types of records
            self.order_record[date] = []    
            self.order_received[date] = []
            self.transfer_record[date] = []
            self.pnl[date] = {'open':{'transfer':0, 'trade':0 ,'book':0, 'update':0, 'index':0}, 
                              'close':{'transfer':0, 'trade':0 ,'book':0, 'update':0, 'index':0}}
            self.fee[date] = {'open': 0, 
                              'close':0}
            self.capital[date] = {}
            self.portfolio_record[date] = {}
        
            for time in self.times:
                self.updatePortfolio(date, time) # update value and return in portfolioportfolio
                self.checkPortfolio(date, time)  # check portfolio holdings status
                strategy.sendOrder(date, time)   # call strategy<class> sendOrder function
                self.updatePNL(date, time)       # update pnl

        
    def checkPortfolio(self, date, time):
        """
        check the portfolio, sell the bond(and stock) if in the last trading day
        """
        for bond_ticker in list(self.portfolio['bond'].keys()):
            if self.isLastDate(date, bond_ticker) and self.isValidTicker(date, bond_ticker, 'bond', time):
                self.clearTicker(date, bond_ticker, asset_type='bond', time=time)
                
                stock_ticker = self.getCbondStock(bond_ticker)  # also sell the stock if holds
                if stock_ticker in self.portfolio['stock'].keys() and self.isValidTicker(date, stock_ticker, 'stock', time):
                    self.clearTicker(date, stock_ticker, asset_type='stock', time=time)
        
    
    def updatePortfolio(self, date, time):
        """
        update the time, value and return in portfolio
        """
        for asset_type in ['bond', 'stock']:
            for ticker in list(self.portfolio[asset_type].keys()):
                
                self.portfolio[asset_type][ticker]['time'] = time 
                
                # add 1 to the holding period in portfolio
                if time == 'open':
                    self.portfolio[asset_type][ticker]['count'] += 1
                    
                    # calculate lending cost for each short position in portfolio
                    if self.portfolio[asset_type][ticker]['amount'] < 0:
                        self.fee[date][time] += np.abs(self.portfolio[asset_type][ticker]['cost']) * self.fee_rate['lending'] / self.annual_date
                
                # update portfolio information if the ticker is tradable today
                if self.isTradeDate(date, ticker, asset_type):
                    price = self.getPrice(date, ticker, asset_type, time)
                    self.portfolio[asset_type][ticker]['value'] = self.portfolio[asset_type][ticker]['amount'] * price
                    self.portfolio[asset_type][ticker]['pnl'] = self.portfolio[asset_type][ticker]['value'] - self.portfolio[asset_type][ticker]['cost']
        
        self.portfolio_record[date][time] = copy.deepcopy(self.portfolio) # record historical portfolio 
        self.pnl[date][time]['update'] += self.getBookPNL(date, time)     # record the change of holding pnl
        self.pnl[date][time]['index'] += self.getTotalCost() * self.getIndexRet(date, time)  # hedge net long position in portfolio by top1500 bond index
        
        
    def updatePNL(self, date, time):
        """
        update P&L after trading
        """                  
        self.capital[date][time] = self.getTotalValue()                   # current total value(cash + asset)
        self.portfolio_record[date][time] = copy.deepcopy(self.portfolio) # record historical portfolio
        self.pnl[date][time]['book'] += self.getBookPNL(date, time)       # record the change of holding pnl
        
        
        
    # ===================================== Analysis Part =======================================
    def calTable(self):
        """
        get DataFrame from backtest historical records
        """
        # pnl table
        pnls = []
        for date in self.pnl.keys():
            pnl = pd.DataFrame(data=self.pnl[date]).unstack().reset_index()
            pnl.columns = ['time', 'pnl_type', 'pnl']
            pnl['date'] = date
            pnls.append(pnl)

        pnl_detail = pd.concat(pnls).reset_index(drop=True) 
        pnl_detail['pnl'] /= self.cap
        
        trans_pnl = pnl_detail[pnl_detail['pnl_type']=='transfer'].groupby(
            ['date', 'time'])['pnl'].sum().unstack().rename(columns={'open':'trans_open', 'close':'trans_close'})
        trade_pnl = pnl_detail[pnl_detail['pnl_type']=='trade'].groupby(
            ['date', 'time'])['pnl'].sum().unstack().rename(columns={'open':'trade_open', 'close':'trade_close'})
        book_pnl = pnl_detail[pnl_detail['pnl_type']=='book'].groupby(
            ['date', 'time'])['pnl'].sum().unstack().rename(columns={'open':'book_open', 'close':'book_close'})
        update_pnl = pnl_detail[pnl_detail['pnl_type']=='update'].groupby(
            ['date', 'time'])['pnl'].sum().unstack().rename(columns={'open':'update_open', 'close':'update_close'})
        index_pnl = pnl_detail[pnl_detail['pnl_type']=='index'].groupby(
            ['date', 'time'])['pnl'].sum().unstack().rename(columns={'open':'index_open', 'close':'index_close'})
        
        # Special case: intra + ovnt is open(T) to open(T+1)，consistent with index ret. 
        # Other columns with 'open': close(T - 1) to open(T)
        pnl_table = pd.concat([trans_pnl, trade_pnl, book_pnl, update_pnl, index_pnl], axis=1)
        pnl_table['pnl_trans'] = pnl_table['trans_open'] + pnl_table['trans_close']
        pnl_table['pnl_trade'] = pnl_table['trade_open'] + pnl_table['trade_close']
        pnl_table['pnl_book'] = pnl_table['book_open'] + pnl_table['book_close']
        pnl_table['pnl_index'] = pnl_table['index_open'].shift(-1).fillna(0) + pnl_table['index_close']
        pnl_table['pnl_open'] = pnl_table['trans_open'] + pnl_table['trade_open'] + pnl_table['book_open']
        pnl_table['pnl_close'] = pnl_table['trans_close'] + pnl_table['trade_close'] + pnl_table['book_close']
        pnl_table['pnl_intra'] = pnl_table['update_close']
        pnl_table['pnl_ovnt'] = pnl_table['pnl_trans'] - (
            pnl_table['update_close'] - pnl_table['book_close'] - pnl_table['trade_close']) + pnl_table['update_open'].shift(-1).fillna(0)
        pnl_table['pnl'] = pnl_table['pnl_intra'] + pnl_table['pnl_ovnt']
        pnl_table['pnl_hedge'] = pnl_table['pnl'] - pnl_table['pnl_index']
        pnl_table['pnl_check'] = pnl_table['pnl_trans'] + pnl_table['pnl_trade'] + pnl_table['pnl_book']
        
        cum_pnl_table = pnl_table.cumsum()
        cum_pnl_table.columns.name=None
        
        check_diff = cum_pnl_table['pnl'].iloc[-1] - cum_pnl_table['pnl_check'].iloc[-1]
        # pnl = pnl_intra + pnl_ovnt, pnl_check = pnl_trans + pnl_trade + pnl_book. The cumulative value of two methods should be same at the end.
        if np.abs(check_diff) > 0.0001:
            print('pnl Warning! pnl:{}, pnl check:{}'.format(cum_pnl_table['pnl'].iloc[-1], cum_pnl_table['pnl_check'].iloc[-1]))
        
        # fee table
        fee_table = pd.DataFrame(data=self.fee.values(), index=self.fee.keys()) / self.cap
        fee_table.rename(columns={'open': 'fee_open', 'close':'fee_close'}, inplace=True)
        fee_table['fee'] = fee_table['fee_open'] + fee_table['fee_close']

        cum_fee_table = fee_table.cumsum()
        
        # net pnl table
        net_table =  pd.merge(left=pnl_table, right=fee_table, left_index=True, right_index=True)
        net_table['net_open'] = net_table['pnl_open'] - net_table['fee_open']
        net_table['net_close'] = net_table['pnl_close'] - net_table['fee_close']
        net_table['net_intra'] = net_table['pnl_intra'] - net_table['fee_open'] - net_table['fee_close']
        net_table['net_ovnt'] = net_table['pnl_ovnt']
        net_table['net'] = net_table['pnl'] - net_table['fee']
        net_table['net_hedge'] = net_table['pnl_hedge'] - net_table['fee']

        cum_net_table = net_table.cumsum()
        
        # capital table
        capital_table = pd.DataFrame(data=self.capital.values(), index=self.capital.keys()) / self.cap
        capital_table.index.name = 'date'
        
        # order table
        orders = []
        for date in self.order_record.keys():
            order = pd.DataFrame(data=self.order_record[date], index=range(len(self.order_record[date])))
            orders.append(order)

        order_table = pd.concat(orders).reset_index(drop=True)
        
        order_count = order_table.groupby(['date', 'dir', 'asset_type'])['ticker'].count().unstack().unstack()
        
        longshort_dict = {1:'long', -1:'short'}
        order_count.columns = ['{}_{}'.format(col[0], longshort_dict[col[1]]) for col in order_count.columns]
        format_cols = ['bond_long', 'bond_short', 'stock_long', 'stock_short']
        new_cols = [col for col in format_cols if col not in order_count.columns]
        order_count[new_cols] = 0
        order_count.fillna(0, inplace=True)

        order_count['bond'] = order_count['bond_long'] + order_count['bond_short']
        order_count['stock'] = order_count['stock_long'] + order_count['stock_short']
        order_count['total'] = order_count['bond'] + order_count['stock']
        
        order_cost = order_table.groupby(['date', 'dir', 'asset_type'])['cost'].sum().unstack().unstack() / self.cap
        
        order_cost.columns = ['{}_{}'.format(col[0], longshort_dict[col[1]]) for col in order_cost.columns]
        new_cols = [col for col in format_cols if col not in order_cost.columns]
        order_cost[new_cols] = 0
        order_cost.fillna(0, inplace=True)

        order_cost['bond'] = order_cost['bond_long'] + order_cost['bond_short']
        order_cost['stock'] = order_cost['stock_long'] + order_cost['stock_short']
        order_cost['total'] = order_cost['bond'] + order_cost['stock']
        
        order_pnl = order_table.loc[order_table['label'].isin(['clean', 'reduce']), 
                                    ['asset_type', 'trade_pnl']].copy()
        
        def winRate(pnl, column):
            """
            calculate winning rate of given pnl series
            """
            if len(pnl) > 0:
                return len(pnl[pnl[column] > 0]) / len(pnl)
            else:
                return 0

        bond_rate = winRate(order_pnl[order_pnl['asset_type']=='bond'], 'trade_pnl')
        stock_rate = winRate(order_pnl[order_pnl['asset_type']=='stock'], 'trade_pnl')
        total_rate = winRate(order_pnl, 'trade_pnl')
        
        # receive table
        receive = []
        for date in self.order_received.keys():
            rec = pd.DataFrame(data=self.order_received[date], index=range(len(self.order_received[date])))
            receive.append(rec)
        receive_table = pd.concat(receive).reset_index(drop=True)
        
        receive_count = receive_table.groupby(['date', 'asset_type'])['ticker'].count().unstack()
        format_cols = ['bond', 'stock']
        new_cols = [col for col in format_cols if col not in receive_count.columns]
        receive_count[new_cols] = 0
        receive_count['total'] = receive_count['stock'] + receive_count['bond']
        
        # transfer(convert) table
        trans = []
        for date in self.transfer_record.keys():
            tran = pd.DataFrame(data=self.transfer_record[date], index=range(len(self.transfer_record[date])))
            trans.append(tran)

        trans = pd.concat(trans).reset_index(drop=True)
        trans_rate = winRate(trans, 'transfer_pnl')
        
        # portfolio table
        port_table = []

        for date in self.portfolio_record.keys():
            for time in ['open', 'close']:
                bond_num, bond_long, bond_short = self.portAnalysis(self.portfolio_record[date][time]['bond'])
                stock_num, stock_long, stock_short = self.portAnalysis(self.portfolio_record[date][time]['stock'])

                port_table.append({'date': date, 'time':time, 
                                  'bond_num':bond_num, 'bond_long':bond_long, 'bond_short':bond_short,
                                  'stock_num':stock_num, 'stock_long': stock_long, 'stock_short':stock_short})

        port_table = pd.DataFrame(port_table)
        port_table.set_index('date', inplace=True)
        
        port_table['num'] = port_table['bond_num'] + port_table['stock_num']
        port_table['long'] = port_table['bond_long'] + port_table['stock_long']
        port_table['short'] = port_table['bond_short'] + port_table['stock_short']
        
        port_table['bond_long_ratio'] = (port_table['bond_long'] / (port_table['bond_long'] - port_table['bond_short'])).fillna(0.5)
        port_table['stock_long_ratio'] = (port_table['stock_long'] / (port_table['stock_long'] - port_table['stock_short'])).fillna(0.5)
        port_table['long_ratio'] = (port_table['long'] / (port_table['long'] - port_table['short'])).fillna(0.5)
        
        port_table['bond_value'] = (port_table['bond_long'] - port_table['bond_short']) / self.cap
        port_table['stock_value'] = (port_table['stock_long'] - port_table['stock_short']) / self.cap
        port_table['value'] = (port_table['long'] - port_table['short']) / self.cap
        
        port_longshort = port_table[['time', 'long_ratio']].reset_index()
        port_longshort = port_longshort.pivot_table(index='date', columns='time', values='long_ratio').fillna(0.5)
        port_longshort.columns.name = None
        
        # save tables in self.table
        self.table = {
            'pnl': pnl_table,
            'cum_pnl':cum_pnl_table,
            'fee': fee_table,
            'cum_fee':cum_fee_table,
            'net': net_table,
            'cum_net':cum_net_table,
            'capital':capital_table,
            'order':order_table,
            'order_count':order_count,
            'order_cost':order_cost,
            'order_pnl': order_pnl,
            'receive':receive_table,
            'receive_count':receive_count,
            'trans':trans,
            'win_rate' : {'bond': bond_rate, 'stock':stock_rate, 'total':total_rate, 'trans':trans_rate},
            'portfolio':port_table,
            'portfolio_longshort':port_longshort,
        }

        
    def pnlAnalysis(self, pnl, cum_pnl, column='performance'):
        """
        return pnl statistics table
        includes return, volatility, sharpe ratio, winning rate and max drawdown
        """
        nav = cum_pnl + 1
        total_return = cum_pnl[-1]
        annual_return = cum_pnl[-1] / len(nav) * self.annual_date 
        annual_volatility = pnl.std() * np.sqrt(self.annual_date)
        riskfree_rate = self.riskfree_rate / self.annual_date
        if annual_volatility == 0:
            sharpe_ratio = 0
        else:
            sharpe_ratio = (pnl.mean() - riskfree_rate) / pnl.std(ddof=1) * np.sqrt(self.annual_date)
        win_rate = (pnl >= 0).sum() / len(pnl)
        max_drawdown = (nav / nav.cummax() - 1).min()

        table = {}
        table['total_return'] = total_return
        table['annual_return'] = annual_return
        table['annual_volatility'] = annual_volatility
        table['sharpe_ratio'] = sharpe_ratio
        table['win_rate'] = win_rate
        table['max_drawdown'] = max_drawdown
        table = pd.DataFrame(data=table.values(), columns=[column], index=table.keys()).T

        return table


    def portAnalysis(self, portfolio):
        """
        helper function used in calTable
        return holding tickers number, long position value and short position value of given portfolio
        """
        num = len(portfolio.keys())
        long_value = 0
        short_value = 0

        if num == 0:
            return num, long_value, short_value
        else:
            value = [portfolio[tkr]['value'] for tkr in portfolio.keys()]

        long_value = sum([val for val in value if val >= 0])
        short_value = sum([val for val in value if val < 0])

        return num, long_value, short_value


    def plot(self):
        """
        Backtest plot function:
        visulized analysis of strategy performance
        """
        cum_pnl_table = self.table['cum_pnl']
        cum_net_table = self.table['cum_net']
        capital_table = self.table['capital']
        order_count = self.table['order_count']
        order_cost = self.table['order_cost']
        port_longshort = self.table['portfolio_longshort']
        
        # part 1: pnl view
        fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2, sharex=True, figsize=(16, 8))
        
        cum_pnl_table[['pnl_intra', 'pnl_ovnt','pnl', 'pnl_hedge']].plot(ax=ax1, linewidth=1.5)
        ax1.set_title('gross pnl')

        cum_net_table[['net_intra', 'net_ovnt','net', 'net_hedge']].plot(ax=ax2, linewidth=1.5)
        ax2.set_title('net pnl')
        
        cum_pnl_table['gross_drawdown'] = (cum_pnl_table['pnl'] + 1) / (cum_pnl_table['pnl'] + 1).cummax()
        cum_pnl_table['gross_hedge_drawdown'] = (cum_pnl_table['pnl_hedge'] + 1) / (cum_pnl_table['pnl_hedge'] + 1).cummax()
        max_drawdown = min(np.floor(cum_pnl_table['gross_drawdown'].min() * 10) / 10, 0.9)
        cum_pnl_table[['gross_drawdown', 'gross_hedge_drawdown']].plot(
            ax=ax3, alpha=0.7, linewidth=1.5, ylim=[max_drawdown, 1.1])
        ax3.set_title('gross pnl drawdown')
        
        cum_net_table['net_drawdown'] = (cum_net_table['net'] + 1) / (cum_net_table['net'] + 1).cummax()
        cum_net_table['net_hedge_drawdown'] = (cum_net_table['net_hedge'] + 1) / (cum_net_table['net_hedge'] + 1).cummax()
        max_drawdown = min(np.floor(cum_net_table['net_drawdown'].min() * 10) / 10, 0.9)
        cum_net_table[['net_drawdown', 'net_hedge_drawdown']].plot(
            ax=ax4, alpha=0.7, linewidth=1.5, ylim=[max_drawdown, 1.1])
        ax4.set_title('net pnl drawdown')

        plt.suptitle('pnl view')
        fig.tight_layout()
        plt.show()
        
        # part 2: strategy view
        fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 8))

        order_count[['bond_long', 'bond_short', 'stock_long', 'stock_short', 'total']].boxplot(ax=ax1)
        ax1.set_title('order count statistics')
        
        order_cost[['bond_long', 'bond_short', 'stock_long', 'stock_short', 'total']].boxplot(ax=ax2)
        ax2.set_title('order cost statistics')

        port_longshort.plot(ax=ax3)
        ax3.set_title('portfolio long ratio')
        
        y_min = np.floor(min(capital_table['open'].min(), capital_table['close'].min()) * 100) / 100
        y_max = np.ceil(max(capital_table['open'].max(), capital_table['close'].max()) * 100) / 100
        capital_table[['open', 'close']].plot(ax=ax4, ylim=[min(y_min, 0.995), max(y_max, 1.005)])
        ax4.set_title('capital consistency')

        plt.suptitle('strategy view')

        fig.tight_layout()
        plt.show()


    def analysis(self, strategy=None, plot=True):
        """
        conclusive analysis function called out of BT system 
        Output statistics and figures to demonstrate strategy performance
        """
        self.calTable()
        
        # pnl statistics
        pnl_table = self.table['pnl']
        cum_pnl_table = self.table['cum_pnl']
        net_table = self.table['net']
        cum_net_table = self.table['cum_net']

        pnl_stat = pd.concat([
            self.pnlAnalysis(pnl_table['pnl'], cum_pnl_table['pnl'], column='pnl'),
            self.pnlAnalysis(pnl_table['pnl_open'], cum_pnl_table['pnl_open'], column='pnl_open'),
            self.pnlAnalysis(pnl_table['pnl_close'], cum_pnl_table['pnl_close'], column='pnl_close'),
            self.pnlAnalysis(pnl_table['pnl_trade'], cum_pnl_table['pnl_trade'], column='pnl_trade'),
            self.pnlAnalysis(pnl_table['pnl_trans'], cum_pnl_table['pnl_trans'], column='pnl_trans'),
            self.pnlAnalysis(pnl_table['pnl_intra'], cum_pnl_table['pnl_intra'], column='pnl_intra'),
            self.pnlAnalysis(pnl_table['pnl_ovnt'], cum_pnl_table['pnl_ovnt'], column='pnl_ovnt'),
            self.pnlAnalysis(pnl_table['pnl_hedge'], cum_pnl_table['pnl_hedge'], column='pnl_hedge'),
            self.pnlAnalysis(net_table['net'], cum_net_table['net'], column='net'),
            self.pnlAnalysis(net_table['net_hedge'], cum_net_table['net_hedge'], column='net_hedge'),
        ])
        
        if plot:
            display(pnl_stat.round(4))

        # order & portfolio statistics
        date_num = len(self.trade_dates)
        order_table = self.table['order']
        order_count = self.table['order_count']
        order_cost = self.table['order_cost']
        order_pnl = self.table['order_pnl']
        trade_num = len(order_count[order_count['total'] > 0])
        trans = self.table['trans']
        win_rate = self.table['win_rate']
        receive_count = self.table['receive_count']
        port_table = self.table['portfolio']

        daily_stat = {'dates': date_num, 
                      'trade_dates': trade_num / date_num,
                      'order_sum': order_count[['bond', 'stock', 'total']].sum().values,
                      'order_mean': order_count[['bond', 'stock', 'total']].sum().values / trade_num,
                      'cover_num': [order_table.loc[order_table['asset_type']=='bond', 'ticker'].nunique(), 
                                    order_table.loc[order_table['asset_type']=='stock', 'ticker'].nunique(),
                                    order_table['ticker'].nunique()],
                      'trade_win': [win_rate['bond'], win_rate['stock'], win_rate['total']],
                      'conv_win': [win_rate['trans'], win_rate['trans'], win_rate['trans']],
                      'order_cost': order_cost[['bond', 'stock', 'total']].sum().values / trade_num,
                      'refuse_rate': (1 - order_count[['bond', 'stock', 'total']].sum() / receive_count.sum()).fillna(0).values,
                      'turnover': order_cost[['bond', 'stock', 'total']].sum().values / 2 * self.annual_date / date_num,
                      'port_num': port_table[['bond_num', 'stock_num', 'num']].mean().values,
                      'port_value': port_table[['bond_value', 'stock_value', 'value']].mean().values,
                      'long_ratio': port_table[['bond_long_ratio', 'stock_long_ratio', 'long_ratio']].mean().values,
                }

        daily_stat = pd.DataFrame(daily_stat, index=['bond', 'stock', 'total'])
        
        if plot:
            display(daily_stat)
            
        self.result = {'pnl':pnl_stat, 'daily':daily_stat}
        
        # call individualized analysis() function defined in Strategy class if necessary
        if strategy is not None:
            strategy_stat = strategy.analysis()
            self.result['strategy'] = strategy_stat
            
            if plot and strategy_stat is not None:
                display(strategy_stat)

        # plot 
        if plot:
            self.plot()